# 구조적 스트리밍의 기초
- 구조적 스트리밍을 구성하는 핵심 개념과 구조적 스트리밍이 얼마나 사용하기 쉬운지 살펴볼것임

## 구조적 스트리밍의 기초
<img src ="https://spark.apache.org/docs/latest/img/structured-streaming-example-model.png" width=70% />
- 스파크의 <strong>구조적 API</strong>(DataFrame, Dataset, SQL)를 사용함


- 스트림 데이터를 <strong>데이터가 계속해서 추가되는 테이블</strong>처럼 다룬다는 것이 핵심 아이디어


- 스트리밍 잡은 <strong>계속해서 신규 입력 데이터를 확인 및 처리</strong>함


- <strong>내고장성을 보장</strong>하면서 신규 데이터가 유입될 때마다 <strong>효율</strong>적으로 처리 결과 갱신
  - 스트림 처리와 관련된 쿼리 구문이 따로 있지 않음(배치와 동일)
    - 그냥 쿼리 실행 유형만 스트리밍으로 지정하면 됨
    - 일부 제약은 있음
  - 내부적으로 사용자의 쿼리를 어떻게 증분할지 자동으로 파악함
  
  
- 스파크의 다른 기능들과 통합해서 데이터에 실시간으로 반응하는 <strong>통합 빅데이터 처리 애플리케이션</strong>을 만들 수 있음
  - 이로 인해 <strong>하나의 프레임워크</strong>를 이용해서 전체 애플리케이션을 쉽게 정의하고 <strong>여러 처리 영역에서 일관된 결과</strong>를 얻을 수 있도록 함
  - ex) 사용자가 스파크 SQL을 통해 대화형으로 조회하는 테이블을 계속해서 갱신
  - ex) MLlib을 사용해 학습한 머신러닝 모델을 적용한 결과를 제공
  - ex) 스파크 데이터소스의 오프라인 데이터와 스트림을 조인

## 핵심 개념

### 트랜스포메이션과 액션
- <strong>기존 트랜스포메이션 개념과 거의 같음</strong>

- 근데 몇 가지 제약사항은 있음
  - 엔진에서 증분 처리를 할 수 없는 일부 쿼리 유형

- 액션은 단 한 가지
  - 스트림 처리르 시작한 뒤 <strong>연속적으로 처리해 결과를 출력하는 액션만</strong> 있음

### 입력 소스
- 스트리밍 방식으로 데이터를 읽을 수 있는 몇 가지 입력 소스(버전 3.1)
  - HDFS나 S3 등 분산 파일 시스템의 파일
  - 카프카
  - 테스팅용 소켓 소스
  - 테스팅용 레이트 소스
    - 특정 개수의 row를 매초마다 생성

### 싱크
- 싱크로 스트림의 <strong>결과를 저장할 목적지</strong>를 명시함

- 싱크와 실행 엔진은 데이터 처리의 진행 상황을 신뢰도 있고 정확하게 추적하는 역할을 함

- 지원하는 출력용 싱크(버전 3.1)
  - 거의 모든 파일 포맷
  - 카프카
  - 출력 레코드에 임의 연산을 실행하는 foreach 싱크
  - 디버깅용 콘솔 싱크
  - 디버깅용 메모리 싱크

### 출력 모드
- 싱크를 정의하기 위해서는 데이터를 출력하는 방법도 정의해야함

- 지원하는 출력 모드(버전 3.1)
  - append: 싱크에 신규 레코드만 추가(default)
  - update: 변경 대상 레코드 자체를 갱신
  - complete: 전체 출력 내용 재작성
  
- 특정 쿼리와 싱크는 일부 출력모드만 지원함
  - ex) 스트림에 map연산만 하는 잡
    - 매번 전체 데이터를 신규 파일로 저장하는 complete 모드는 적합하지 않음

### 트리거
- 트리거는 <strong>데이터 출력 시점</strong>을 정의함
  - 언제 신규 데이터를 확인하고 결과를 갱신할지 정의함
  
- 처리 시간(고정된 주기로만 신규 데이터 탐색)기반의 트리거도 지원

### 이벤트 시간 처리
- 이벤트 시간 기준의 처리도 지원함

- 이 처리 방식은 무작위로 도착한 <strong>레코드 내부에 기록된 타임스탬프</strong>를 기준으로함

- 두 가지 핵심 아이디어
  - <strong>이벤트 시간 데이터</strong>
    - 이벤트 시간은 데이터에 기록된 시간 필드를 의미
    - 표준 SQL 연산자를 이용해 그룹화, 집계, 윈도우 처리를 하여 이벤트 시간 필드를 인식하도록 해서 이벤트 시간 처리 가능
    
  - <strong>워터마크</strong>
    - 시간 제한을 설정할 수 있는 스트리밍 시스템 기능
    - 늦게 들어온 이벤트를 어디까지 처리할지 시간을 제한할 수 있음